# Data collection for Optimsation Code


Before running code, pvlib will need to be installed in the terminal. This is done with the comand:
pip install pvlib


Import necessary modules


In [14]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import pvlib
from pvlib import pvsystem
from pathlib import Path
import requests
import math
import glob
from datetime import timedelta

In [2]:
%run ./variables.ipynb

⚠️ Warning: Blank cells detected in the *Input Data* tab (first row):

  → Column 'Available South Facing Roof Area for Panels (m2)' is blank
  → Column 'Available East Facing Roof Area for Panels (m2)' is blank
  → Column 'Available West Facing Roof Area for Panels (m2)' is blank
Blank cells set to zero


/opt/anaconda3/lib/python3.12/site-packages/openpyxl/worksheet/_read_only.py:85: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():
/opt/anaconda3/lib/python3.12/site-packages/openpyxl/worksheet/_read_only.py:85: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


Configuration variables

In [3]:
#site_coords = (-35.7333, 143.9167)  # East Kerang coordinates (lat, lon)
#start_year = 2023
end_year = start_year
start_month = 1
peak_hours = [(15, 21)]  # Peak period: 3pm-9pm
pv_type = 'roof_flush'  # or 'roof_tilt'

# Spot market configuration
#nem_state = 'VIC1'  # VIC1, NSW1, QLD1, SA1, TAS1
download_spot_prices = True

# # Electricity Bill Charges (monthly)
# elec_bill_charges = [
#     0.66528+0.6456+1.15714+0.0294+0.55125+10.62,      # Jan
#     0.64476+0.5556+1.15714+0.01262+0.05438+10.62,     # Feb
#     0.75768+0.8504+1.14330+0.01405+0.04326+0.00987+9.81,  # Mar
#     0.75768+0.8504+1.14330+0.01663+0.04364+9.81,      # Apr
#     0.75768+0.8504+1.14330+0.08053+0.04343+9.81,      # May
#     0.75768+0.8504+1.14330+0.01790+0.04333+9.81,      # Jun
#     0.75768+0.8504+1.14330+0.01460+0.05510+10.62,     # Jul
#     0.75768+0.8504+1.14330+0.02245+0.05561+10.62,     # Aug
#     0.75768+0.8504+1.14330+0.02018+0.05587+10.62,     # Sep
#     0.75768+0.8504+1.14330+0.02450+0.05635+10.62,     # Oct
#     0.75768+0.8504+1.14330+0.02282+0.05588+10.62,     # Nov
#     0.75768+0.8504+1.14330+0.02319+0.05575+0.11172+10.62  # Dec
# ]

# Setup output directory structure
base_dir = Path.cwd()
data_dir = base_dir / 'data'
data_dir.mkdir(exist_ok=True)

# Create year-specific folder
if start_year == end_year:
    year_dir = data_dir / str(start_year)
else:
    year_dir = data_dir / f"{start_year}_{end_year}"
year_dir.mkdir(exist_ok=True)

#### File analysis functions

In [4]:
def file_exists(filepath):
    """Check if file exists and return boolean."""
    exists = Path(filepath).exists()
    if exists:
        print(f"File already exists: {filepath}")
    return exists


def resample_to_5min(df, timestamp_col='Datetime'):
    """Resample DataFrame to 5-minute intervals using forward fill."""
    df[timestamp_col] = pd.to_datetime(df[timestamp_col])
    df = df.drop_duplicates(subset=[timestamp_col], keep='first').set_index(timestamp_col)

    inferred_freq = pd.infer_freq(df.index)
    if inferred_freq in ['5min', '5T']:
        return df.reset_index()

    print(f"Detected frequency: {inferred_freq}. Resampling to 5min...") 

    # Extend the index by one more expected period (assuming hourly)
    last_time = df.index[-1]
    next_time = last_time + pd.Timedelta(hours=1)
    df_extended = pd.concat([df, pd.DataFrame(index=[next_time])])

    df_resampled = df_extended.resample('5min').ffill()
    
    # Drop the final (midnight) row and restore original column name
    df_resampled = df_resampled.iloc[:-1].reset_index()
    df_resampled.rename(columns={'index': timestamp_col}, inplace=True)

    return df_resampled

# def resample_to_5min_kw(df, timestamp_col, data_cols):
#     """
#     Resample hourly kW data to 5-minute intervals by dividing by 12.
    
#     Parameters:
#     -----------
#     df : pd.DataFrame
#         Input dataframe with hourly data
#     timestamp_col : str
#         Name of timestamp column
#     data_cols : list
#         List of column names to resample
#     """
#     df[timestamp_col] = pd.to_datetime(df[timestamp_col])
#     print("LOOK HERRRREE")
#     print(df[[timestamp_col]].head(5))
#     data_5min = []
    
#     for _, row in df.iterrows():
#         timestamp = row[timestamp_col]
#         per_5min_values = {col: row[col] / 12 for col in data_cols}
        
#         for i in range(12):
#             new_timestamp = timestamp + pd.Timedelta(minutes=5*i)
#             new_row = {'timestamp': new_timestamp}
#             new_row.update(per_5min_values)
#             data_5min.append(new_row)
    
#     df_5min = pd.DataFrame(data_5min)
#     df_5min['timestamp'] = df_5min['timestamp'].dt.strftime('%-d/%-m/%Y %-H:%M')
#     return df_5min

Resample dataframe 5 min

In [5]:
def resample_to_5min_kw(df, timestamp_col, data_cols):
    """
    Resample hourly kW data to 5-minute intervals by dividing by 12.
    
    Parameters:
    -----------
    df : pd.DataFrame
        Input dataframe with hourly data
    timestamp_col : str
        Name of timestamp column
    data_cols : list or str
        List of column names to resample, or single column name as string
    """
    # Convert single string to list
    if isinstance(data_cols, str):
        data_cols = [data_cols]
    
    df[timestamp_col] = pd.to_datetime(df[timestamp_col])
    print(df[[timestamp_col]].head(5))
    data_5min = []
    
    for _, row in df.iterrows():
        timestamp = row[timestamp_col]
        per_5min_values = {col: row[col] / 12 for col in data_cols}
        
        for i in range(12):
            new_timestamp = timestamp + pd.Timedelta(minutes=5*i)
            new_row = {'timestamp': new_timestamp}
            new_row.update(per_5min_values)
            data_5min.append(new_row)
    
    df_5min = pd.DataFrame(data_5min)
    df_5min['timestamp'] = df_5min['timestamp'].dt.strftime('%-d/%-m/%Y %-H:%M')
    return df_5min


#### Grid/Retail Price Generation Function

In [6]:
def generate_grid_prices(prices, peak_hours, start_year, end_year, start_month=1):
    """Generate hourly grid prices with peak/off-peak rates."""
    if start_year == end_year:
        filename = year_dir / f"grid_prices_{start_year}.csv"
    else:
        filename = year_dir / f"grid_prices_{start_year}_{end_year}.csv"
    
    if file_exists(filename):
        return pd.read_csv(filename)
    
    all_data = []
    
    for year in range(start_year, end_year + 1):
        first_month = start_month if year == start_year else 1
        start_date = f"{year}-{first_month:02d}-01 00:00"
        end_date = pd.Period(f"{year}-12", freq="M").end_time.replace(hour=23)
        
        date_range = pd.date_range(start=start_date, end=end_date, freq="h")
        
        hourly_prices = []
        for dt in date_range:
            month_idx = dt.month - 1
            peak_price, offpeak_price = prices[month_idx]
            is_peak = any(start <= dt.hour < end for start, end in peak_hours)
            hourly_prices.append(peak_price if is_peak else offpeak_price)
        
        year_df = pd.DataFrame({
            "Datetime": date_range,
            "Price ($/kWh)": hourly_prices
        })
        all_data.append(year_df)
    
    df = pd.concat(all_data, ignore_index=True)
    df.to_csv(filename, index=False)
    print(f"Created: {filename}")
    return df



# test_retail = generate_grid_prices(prices, peak_hours, start_year, end_year, start_month=1)

# test_retail = resample_to_5min(test_retail)

# print(test_retail.tail(10))


#### Solar Generation Functions



In [7]:
def get_weather_data(lat, lon, year, start_year=2005, end_year=2023):
    """Fetch TMY weather data from PVGIS."""
    all_data = pvlib.iotools.get_pvgis_tmy(
        float(lat), float(lon),
        startyear=start_year, endyear=end_year,
        outputformat='json', map_variables=True
    )
    
    data = all_data[0].reset_index()
    data['time(UTC)'] = pd.to_datetime(data['time(UTC)'])
    data['time(UTC)'] = data['time(UTC)'].apply(lambda x: x.replace(year=year))
    data = data.set_index('time(UTC)')
    return data


def calculate_pv_profile(weather_data, latitude, longitude, surface_azimuth, 
                        surface_tilt, type_pv='roof_flush'):
    """Calculate PV generation profile (kW per kW installed)."""
    temperature = weather_data['temp_air'].astype(float)
    windspeed = weather_data['wind_speed'].astype(float)
    ghi = weather_data['ghi']
    dni = weather_data['dni']
    dhi = weather_data['dhi']
    pressure = weather_data['pressure'].astype(float)
    
    sp = pvlib.solarposition.get_solarposition(
        weather_data.index, float(latitude), float(longitude), pressure=pressure
    )
    
    irradiance = pvlib.irradiance.get_total_irradiance(
        surface_tilt, surface_azimuth, sp.apparent_zenith, sp.azimuth,
        dni, ghi, dhi
    )
    
    modules = pvsystem.retrieve_sam('SandiaMod').T
    modules_multi_si = modules[modules['Material'] == 'mc-Si']
    multi_si = modules_multi_si[
        modules_multi_si.index.str.contains('Sharp_NDQ235F4__2013_')
    ].T.iloc[:, 0]
    
    if type_pv == 'roof_flush':
        temp_model_params = pvlib.temperature.TEMPERATURE_MODEL_PARAMETERS['sapm']['close_mount_glass_glass']
    elif type_pv == 'roof_tilt':
        temp_model_params = pvlib.temperature.TEMPERATURE_MODEL_PARAMETERS['sapm']['open_rack_glass_polymer']
    else:
        raise ValueError('Invalid type of PV panel installation')
    
    celltemp = pvlib.temperature.sapm_cell(
        irradiance.poa_global, temperature, windspeed, **temp_model_params
    )
    
    airmass = pvlib.atmosphere.get_relative_airmass(sp.apparent_zenith)
    airmass_absolute = pvlib.atmosphere.get_absolute_airmass(airmass, pressure=pressure)
    aoi = pvlib.irradiance.aoi(surface_tilt, surface_azimuth, sp.zenith, sp.azimuth)
    
    effective_irradiance = pvlib.pvsystem.sapm_effective_irradiance(
        irradiance.poa_direct, irradiance.poa_diffuse,
        airmass_absolute, aoi, multi_si
    )
    
    dc = pvlib.pvsystem.sapm(effective_irradiance, celltemp, multi_si)
    kWp = (multi_si.Impo * multi_si.Vmpo) / 1000
    
    sapm_inverters = pvlib.pvsystem.retrieve_sam('cecinverter')
    inverter = sapm_inverters['ABB__MICRO_0_25_I_OUTD_US_208__208V_']
    ac = pvlib.inverter.sandia(dc['v_mp'], dc['p_mp'], inverter)
    
    ac_mod = ac.fillna(0) / 1000
    ac_mod[ac_mod < 0] = 0
    profile = (1/kWp) * ac_mod
    
    return profile


def generate_directional_solar_profiles(lat, lon, start_year, end_year, type_pv='roof_flush'):
    """Generate solar profiles for all four cardinal directions."""
    if start_year == end_year:
        filename = year_dir / f"solar_hourly_generation_{start_year}.csv"
    else:
        filename = year_dir / f"solar_hourly_generation_{start_year}_{end_year}.csv"
    
    if file_exists(filename):
        return pd.read_csv(filename)
    
    directions = {'north': 0, 'east': 90, 'south': 180, 'west': 270}
    tilt = round(abs(lat))
    
    all_data = []
    for year in range(start_year - 1, end_year + 1):
        year_weather_data = get_weather_data(lat, lon, year)
        
        year_profiles = {}
        for direction, azimuth in directions.items():
            year_profiles[direction] = calculate_pv_profile(
                year_weather_data, lat, lon, azimuth, tilt, type_pv
            )
        
        year_df = pd.DataFrame({
            'timestamp': year_weather_data.index,
            'north_solar_capacity_kw_per_kw': year_profiles['north'].values,
            'east_solar_capacity_kw_per_kw': year_profiles['east'].values,
            'south_solar_capacity_kw_per_kw': year_profiles['south'].values,
            'west_solar_capacity_kw_per_kw': year_profiles['west'].values
        })
        all_data.append(year_df)
    
    results_df = pd.concat(all_data, ignore_index=True)
    results_df.to_csv(filename, index=False)
    print(f"Created: {filename}")
    return results_df

#### Spot Market Functions

In [8]:
def download_spot_prices(year, month, state='VIC1'):
    """Download AEMO spot market prices for a given year and month."""
    filename = year_dir / f'spot_prices_{year}_{month}_{state}.csv'
    
    if file_exists(filename):
        return filename
    
    url = f'https://www.aemo.com.au/aemo/data/nem/priceanddemand/PRICE_AND_DEMAND_{year}{month}_{state}.csv'
    
    try:
        response = requests.get(url, stream=True)
        response.raise_for_status()
        
        with open(filename, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        
        return filename
    except Exception as e:
        print(f"Error downloading spot prices: {e}")
        return None


def download_spot_prices_batch(years, months, state='VIC1', combine_per_year=True):
    """Download multiple months of spot price data and combine by year."""
    if isinstance(years, int):
        years = [years]
    
    if not combine_per_year:
        downloaded_files = []
        for year in years:
            for month in months:
                file_path = download_spot_prices(year, month, state)
                if file_path:
                    downloaded_files.append(file_path)
        return downloaded_files
    
    combined_files = []
    for year in years:
        combined_filename = year_dir / f'spot_prices_{year}_{state}.csv'
        
        if file_exists(combined_filename):
            combined_files.append(combined_filename)
            continue
        
        all_data = []
        for month in months:
            file_path = download_spot_prices(year, month, state)
            if file_path:
                df = pd.read_csv(file_path)
                all_data.append(df)
                os.remove(file_path)
        
        if all_data:
            combined_df = pd.concat(all_data, ignore_index=True)
            combined_df.to_csv(combined_filename, index=False)
            print(f"Created: {combined_filename} ({len(combined_df)} records)")
            combined_files.append(combined_filename)
    
    return combined_files

### Data Proccessing

Generate Synthesized Demand File

In [10]:
def merge_excel_files_with_electricity_sum(folder_path, output_file=None):
    """
    Merges all Excel files in a folder, combining sheets and files by timestamp,
    then sums all numeric columns into a single 'electricity' column.
    
    Parameters:
    -----------
    folder_path : str
        Path to folder containing Excel files
    output_file : str, optional
        Name of output CSV file (default: auto-generated based on data year range)
    
    Returns:
    --------
    pd.DataFrame
        Merged dataframe with electricity column
    """
    
    all_data = []
    
    # Get all Excel files in the folder
    excel_files = list(Path(folder_path).glob('*.xlsx')) + list(Path(folder_path).glob('*.xls'))
    excel_files = [f for f in excel_files if not f.name.startswith('~$')]
    
    if not excel_files:
        print(f"No Excel files found in '{folder_path}'")
        return None
    
    # Process each Excel file
    for file_path in excel_files:
        print(f"\nProcessing: {file_path.name}")
        
        try:
            # Read all sheets from the Excel file
            excel_file = pd.ExcelFile(file_path)
            
            for sheet_name in excel_file.sheet_names:
                print(f"  Reading sheet: {sheet_name}")
                
                # Read the sheet
                df = pd.read_excel(file_path, sheet_name=sheet_name)
                
                # Add source information
                df['source_file'] = file_path.name
                df['source_sheet'] = sheet_name
                
                all_data.append(df)
                
        except Exception as e:
            print(f"  Error reading {file_path.name}: {e}")
            continue
    
    if not all_data:
        print("No data was successfully read from the files")
        return None
    
    # Concatenate all dataframes
    print("\nMerging all data...")
    merged_df = pd.concat(all_data, ignore_index=True)
    
    print(f"Total rows before processing: {len(merged_df)}")
    print(f"Data from {len(excel_files)} file(s) and {len(all_data)} sheet(s)")
    
    # Identify timestamp column (common names)
    timestamp_cols = ['Timestamp', 'timestamp', 'DateTime', 'datetime', 'Date', 'date', 'Time', 'time']
    timestamp_col = None
    
    for col in timestamp_cols:
        if col in merged_df.columns:
            timestamp_col = col
            break
    
    # Convert timestamp to datetime if not already
    if timestamp_col:
        print(f"Found timestamp column: {timestamp_col}")
        # Try to convert to datetime if not already
        try:
            # First try unix timestamp
            if merged_df[timestamp_col].dtype in ['int64', 'float64']:
                merged_df[timestamp_col] = pd.to_datetime(merged_df[timestamp_col], unit='s', errors='coerce')
            else:
                merged_df[timestamp_col] = pd.to_datetime(merged_df[timestamp_col], errors='coerce')
        except:
            pass
    
    # Identify columns to exclude from summation
    exclude_cols = ['Hours', 'Timestamp', 'DateTime', 'Date', 'Time', 
                    'source_file', 'source_sheet', 'Total', 'True Total']
    
    # Find all numeric columns to sum (excluding the ones we want to keep)
    numeric_cols = []
    for col in merged_df.columns:
        if col not in exclude_cols:
            # Check if column is numeric
            if pd.api.types.is_numeric_dtype(merged_df[col]):
                numeric_cols.append(col)
    
    print(f"\nSumming {len(numeric_cols)} numeric columns into 'electricity' column")
    
    # Create electricity column by summing all numeric columns
    merged_df['electricity'] = merged_df[numeric_cols].sum(axis=1)
    
    # Create final dataframe with desired columns
    keep_cols = []
    
    # Keep timestamp-related columns if they exist
    for col in ['Hours', 'Timestamp', 'DateTime']:
        if col in merged_df.columns:
            keep_cols.append(col)
    
    # Add electricity column
    keep_cols.append('electricity')
    
    final_df = merged_df[keep_cols].copy()
    
    # Group by timestamp and sum electricity values from all files/sheets
    if timestamp_col and timestamp_col in final_df.columns:
        print(f"\nGrouping by {timestamp_col} and summing electricity values across all files and sheets...")
        
        # Create aggregation dictionary
        agg_dict = {'electricity': 'sum'}
        
        # Keep first occurrence of other columns (like Hours, DateTime)
        for col in keep_cols:
            if col not in [timestamp_col, 'electricity']:
                agg_dict[col] = 'first'
        
        # Group by timestamp and aggregate
        final_df = final_df.groupby(timestamp_col, as_index=False).agg(agg_dict)
        
        # Sort by timestamp
        final_df = final_df.sort_values(by=timestamp_col).reset_index(drop=True)
        
        print(f"Consolidated from {len(merged_df)} total rows to {len(final_df)} unique timestamps")
        
        # Determine year range from actual data if output_file not specified
        if output_file is None:
            try:
                first_year = final_df[timestamp_col].min().year
                last_year = final_df[timestamp_col].max().year
                
                if first_year == last_year:
                    output_file = f'demand_electricity_{first_year}.csv'
                else:
                    output_file = f'demand_electricity_{first_year}_{last_year}.csv'
                
                print(f"\nDetected data year range: {first_year} to {last_year}")
            except:
                output_file = 'demand_electricity.csv'
                print("\nWarning: Could not determine year range, using default filename")
    else:
        print("\nWarning: No timestamp column found for grouping. Keeping all rows.")
        if output_file is None:
            output_file = 'demand_electricity.csv'
    
    # Save to CSV
    output_path = Path(folder_path) / output_file
    final_df.to_csv(output_path, index=False)
    print(f"\nMerged data saved to: {output_path}")
    print(f"Total rows: {len(final_df)}")
    print(f"\nFirst few rows:")
    print(final_df.head(10))
    
    return final_df

# Specify your folder path
folder_path = 'Upload Demand Files Here' # Change this folder name as needed
    
# Run the merge function
result_df = merge_excel_files_with_electricity_sum(folder_path)
    
print("\nProcess complete!")

No Excel files found in 'Upload Demand Files Here'

Process complete!


Data collection and dataframe defintions

In [11]:
print("GENERATING DATA FILES")


# Generate solar profiles (if not exists)
print("GENERATING SOLAR PROFILES")
solar_df = generate_directional_solar_profiles(
    site_coords[0], site_coords[1], start_year, end_year, pv_type
)

# Generate retail grid prices (if not exists)
print("GENERATING RETAIL PRICES")
prices = [[peak_energy_price + oc, off_peak_energy_price + oc] for oc in elec_bill_charges]
prices = np.array(prices) / 100  # Convert cents to dollars
grid_prices_df = generate_grid_prices(prices, peak_hours, start_year, end_year, start_month)

# Download spot prices (optional)
if download_spot_prices:
    print("DOWNLOADING SPOT PRICES")
    years = list(range(start_year, end_year + 1))
    months = [f'{i:02d}' for i in range(1, 13)]
    spot_files = download_spot_prices_batch(
        years=years, months=months, state=nem_state, combine_per_year=True
    )

# Resample to 5-minute intervals
print("RESAMPLING TO 5-MINUTE INTERVALS")

# Grid prices to 5-min
if start_year == end_year:
    grid_5min_path = year_dir / f"grid_prices_5min_{start_year}.csv"
else:
    grid_5min_path = year_dir / f"grid_prices_5min_{start_year}_{end_year}.csv"

if not file_exists(grid_5min_path):
    grid_5min = resample_to_5min(grid_prices_df, 'Datetime')
    grid_5min.to_csv(grid_5min_path, index=False)
    print(f"Created: {grid_5min_path}")

# Solar to 5-min
if start_year == end_year:
    solar_5min_path = year_dir / f"solar_generation_5min_{start_year}.csv"
else:
    solar_5min_path = year_dir / f"solar_generation_5min_{start_year}_{end_year}.csv"

if not file_exists(solar_5min_path):
    solar_df['timestamp'] = pd.to_datetime(solar_df['timestamp'])
    solar_df['timestamp'] = solar_df['timestamp'].dt.tz_convert('Australia/Brisbane')
    solar_5min = resample_to_5min_kw(
        solar_df,
        timestamp_col='timestamp',
        data_cols=['north_solar_capacity_kw_per_kw', 'east_solar_capacity_kw_per_kw',
                  'south_solar_capacity_kw_per_kw', 'west_solar_capacity_kw_per_kw']
    )
    solar_5min.to_csv(solar_5min_path, index=False)
    print(f"Created: {solar_5min_path}")

# Demand to 5-min (if the hourly file exists in Upload Demand Files Here)
demand_hourly_file = Path(data_dir / 'Upload Demand Files Here') / f'demand_electricity_{start_year}.csv'
if demand_hourly_file.exists():
    if start_year == end_year:
        demand_5min_path = year_dir / f"demand_electricity_5min_{start_year}.csv"
    else:
        demand_5min_path = year_dir / f"demand_electricity_5min_{start_year}_{end_year}.csv"
    
    if not file_exists(demand_5min_path):
        demand_hourly = pd.read_csv(demand_hourly_file)
        demand_5min = resample_to_5min_kw(
            demand_hourly, 
            timestamp_col='Date',
            data_cols=['electricity']
        )
        demand_5min.to_csv(demand_5min_path, index=False)
        print(f"Created: {demand_5min_path}")


print("FILE GENERATION COMPLETE!")
print(f"All files saved to: {year_dir}")

GENERATING DATA FILES
GENERATING SOLAR PROFILES
File already exists: /Users/luellaphillips/eaas1/data/2023/solar_hourly_generation_2023.csv
GENERATING RETAIL PRICES
Created: /Users/luellaphillips/eaas1/data/2023/grid_prices_2023.csv
DOWNLOADING SPOT PRICES
File already exists: /Users/luellaphillips/eaas1/data/2023/spot_prices_2023_VIC1.csv
RESAMPLING TO 5-MINUTE INTERVALS
Detected frequency: h. Resampling to 5min...
Created: /Users/luellaphillips/eaas1/data/2023/grid_prices_5min_2023.csv
File already exists: /Users/luellaphillips/eaas1/data/2023/solar_generation_5min_2023.csv
File already exists: /Users/luellaphillips/eaas1/data/2023/demand_electricity_5min_2023.csv
FILE GENERATION COMPLETE!
All files saved to: /Users/luellaphillips/eaas1/data/2023


In [ ]:

print("DATA COLLECTION AND PROCESSING")
print(f"Output directory: {year_dir}")
print(f"Year range: {start_year} to {end_year}")

# Initialize all dataframes to None
demand_df = None
spot_df = None
grid_prices_df = None
solar_df = None

# Load demand data (5-minute intervals)
print("LOADING DEMAND DATA")

# First, try the expected filename
if start_year == end_year:
    demand_5min_path = year_dir / f"demand_electricity_5min_{start_year}.csv"
else:
    demand_5min_path = year_dir / f"demand_electricity_5min_{start_year}_{end_year}.csv"

if demand_5min_path.exists():
    demand_df = pd.read_csv(demand_5min_path)
    print(f"Loaded demand data: {demand_5min_path}")
    print(f"Shape: {demand_df.shape}")
else:
    print(f"Expected file not found: {demand_5min_path}")
    
    # Try to find ANY demand 5min file in year_dir
    print("Searching for any demand_electricity_5min_*.csv file...")
    demand_files = list(year_dir.glob('demand_electricity_5min_*.csv'))
    
    if demand_files:
        demand_df = pd.read_csv(demand_files[0])
        print(f"Loaded demand data from: {demand_files[0]}")
        print(f"Shape: {demand_df.shape}")
    else:
        print(f"No demand 5min files found in {year_dir}")
        
        # Last resort: try Upload Demand Files Here folder
        testing_folder = Path('Upload Demand Files Here')
        if testing_folder.exists():
            print(f"Searching in {testing_folder}...")
            demand_files = list(testing_folder.glob('demand_electricity_5min_*.csv'))
            if demand_files:
                demand_df = pd.read_csv(demand_files[0])
                print(f"Loaded demand data from: {demand_files[0]}")
                print(f"Shape: {demand_df.shape}")
            else:
                print(f"No demand files found in {testing_folder}")

# Load spot prices (5-minute intervals)
print("LOADING SPOT PRICES")

if start_year == end_year:
    spot_file_path = year_dir / f'spot_prices_{start_year}_{nem_state}.csv'
else:
    spot_file_path = year_dir / f'spot_prices_{start_year}_{end_year}_{nem_state}.csv'

if spot_file_path.exists():
    spot_df = pd.read_csv(spot_file_path)
    print(f"Loaded spot prices: {spot_file_path}")
    print(f"Shape: {spot_df.shape}")
else:
    print(f"Spot price file not found: {spot_file_path}")

# Load retail grid prices (5-minute intervals)
print("LOADING RETAIL GRID PRICES")

if start_year == end_year:
    grid_5min_path = year_dir / f"grid_prices_5min_{start_year}.csv"
else:
    grid_5min_path = year_dir / f"grid_prices_5min_{start_year}_{end_year}.csv"

if grid_5min_path.exists():
    grid_prices_df = pd.read_csv(grid_5min_path)
    print(f"Loaded grid prices: {grid_5min_path}")
    print(f"Shape: {grid_prices_df.shape}")
else:
    print(f"Grid price file not found: {grid_5min_path}")

# Load solar generation profiles (5-minute intervals)
print("LOADING SOLAR GENERATION PROFILES")

if start_year == end_year:
    solar_5min_path = year_dir / f"solar_generation_5min_{start_year}.csv"
else:
    solar_5min_path = year_dir / f"solar_generation_5min_{start_year}_{end_year}.csv"

if solar_5min_path.exists():
    solar_df = pd.read_csv(solar_5min_path)
    print(f"Loaded solar generation: {solar_5min_path}")
    print(f"Shape: {solar_df.shape}")
else:
    print(f"Solar generation file not found: {solar_5min_path}")

# Convert timestamp columns to datetime for all dataframes
print("PROCESSING TIMESTAMPS")

if demand_df is not None:
    timestamp_col = None
    for col in ['timestamp', 'Timestamp', 'DateTime', 'datetime']:
        if col in demand_df.columns:
            timestamp_col = col
            break
    if timestamp_col:
        #demand_df[timestamp_col] = pd.to_datetime(demand_df[timestamp_col], format="%d/%m/%Y %H:%M")
        demand_df[timestamp_col] = pd.to_datetime(demand_df[timestamp_col])
        print(f"Demand timestamp column: {timestamp_col}")

if spot_df is not None:
    if 'SETTLEMENTDATE' in spot_df.columns:
        spot_df['timestamp'] = pd.to_datetime(spot_df['SETTLEMENTDATE'])
        print("Spot prices timestamp column: SETTLEMENTDATE -> timestamp")
    elif 'timestamp' in spot_df.columns:
        spot_df['timestamp'] = pd.to_datetime(spot_df['timestamp'])
        print("Spot prices timestamp column: timestamp")

if grid_prices_df is not None:
    if 'Datetime' in grid_prices_df.columns:
        grid_prices_df['Datetime'] = pd.to_datetime(grid_prices_df['Datetime'])
        print("Grid prices timestamp column: Datetime")
    elif 'timestamp' in grid_prices_df.columns:
        grid_prices_df['timestamp'] = pd.to_datetime(grid_prices_df['timestamp'])
        print("Grid prices timestamp column: timestamp")


if solar_df is not None:
    if 'timestamp' in solar_df.columns:
        #solar_df['timestamp'] = pd.to_datetime(solar_df['timestamp'], format="%d/%m/%Y %H:%M").dt.tz_localize(None)
        solar_df['timestamp'] = pd.to_datetime(solar_df['timestamp']).dt.tz_localize(None)
        print("Solar timestamp column: timestamp")

# Display dataframe summaries
print("DATAFRAME SUMMARIES (5-MINUTE INTERVALS)")

print("\n--- Demand DataFrame ---")
if demand_df is not None:
    print(f"Shape: {demand_df.shape}")
    print(f"Columns: {list(demand_df.columns)}")
    print(demand_df.head())
else:
    print("Not loaded")

print("\n--- Spot Prices DataFrame ---")
if spot_df is not None:
    print(f"Shape: {spot_df.shape}")
    print(f"Columns: {list(spot_df.columns)}")
    print(spot_df.head())
else:
    print("Not loaded")

print("\n--- Grid Prices DataFrame ---")
if grid_prices_df is not None:
    print(f"Shape: {grid_prices_df.shape}")
    print(f"Columns: {list(grid_prices_df.columns)}")
    print(grid_prices_df.head())
else:
    print("Not loaded")

print("\n--- Solar DataFrame ---")
if solar_df is not None:
    print(f"Shape: {solar_df.shape}")
    print(f"Columns: {list(solar_df.columns)}")
    print(solar_df.head())
else:
    print("Not loaded")

print("DATA LOADING COMPLETE!")
print(f"All data loaded from: {year_dir}")

DATA COLLECTION AND PROCESSING
Output directory: /Users/luellaphillips/eaas1/data/2023
Year range: 2023 to 2023
LOADING DEMAND DATA
Loaded demand data: /Users/luellaphillips/eaas1/data/2023/demand_electricity_5min_2023.csv
Shape: (105120, 2)
LOADING SPOT PRICES
Loaded spot prices: /Users/luellaphillips/eaas1/data/2023/spot_prices_2023_VIC1.csv
Shape: (105120, 5)
LOADING RETAIL GRID PRICES
Loaded grid prices: /Users/luellaphillips/eaas1/data/2023/grid_prices_5min_2023.csv
Shape: (105120, 2)
LOADING SOLAR GENERATION PROFILES
Loaded solar generation: /Users/luellaphillips/eaas1/data/2023/solar_generation_5min_2023.csv
Shape: (210240, 5)
PROCESSING TIMESTAMPS
Demand timestamp column: timestamp
Spot prices timestamp column: SETTLEMENTDATE -> timestamp
Grid prices timestamp column: Datetime
Solar timestamp column: timestamp
DATAFRAME SUMMARIES (5-MINUTE INTERVALS)

--- Demand DataFrame ---
Shape: (105120, 2)
Columns: ['timestamp', 'electricity']
            timestamp  electricity
0 2023-01-0

Allign dataframes by set year

In [13]:


def align_dataframes_by_year(demand_df, spot_df, grid_prices_df, solar_df, target_year):
    """
    Align all dataframes to a specific year with matching timestamps at 5-minute intervals.
    
    Parameters:
    -----------
    demand_df : pd.DataFrame
        Demand data with timestamp column
    spot_df : pd.DataFrame
        Spot price data with timestamp column
    grid_prices_df : pd.DataFrame
        Grid/retail price data with timestamp column
    solar_df : pd.DataFrame
        Solar generation data with timestamp column
    target_year : int
        Year to align all dataframes to
    
    Returns:
    --------
    tuple of pd.DataFrame
        (aligned_demand, aligned_spot, aligned_grid, aligned_solar)
    """
    
    print(f"ALIGNING DATAFRAMES TO YEAR {target_year}")
    
    # Create reference timestamp range for the target year (5-minute intervals)
    start_date = pd.Timestamp(f'{target_year}-01-01 00:00:00')
    end_date = pd.Timestamp(f'{target_year}-12-31 23:55:00')
    reference_timestamps = pd.date_range(start=start_date, end=end_date, freq='5min')
    
    print(f"\nReference timestamp range:")
    print(f"  Start: {reference_timestamps[0]}")
    print(f"  End: {reference_timestamps[-1]}")
    print(f"  Total intervals: {len(reference_timestamps)}")
    
    # Create reference dataframe
    aligned_data = pd.DataFrame({'timestamp': reference_timestamps})
    
    # Function to identify timestamp column
    def find_timestamp_col(df):
        for col in ['timestamp', 'Timestamp', 'DateTime', 'Datetime', 'datetime', 'SETTLEMENTDATE']:
            if col in df.columns:
                return col
        return None
    
    # Function to remove timezone info
    def remove_timezone(timestamp_series):
        """Remove timezone information from timestamp series"""
        if hasattr(timestamp_series.dtype, 'tz') and timestamp_series.dtype.tz is not None:
            return timestamp_series.dt.tz_localize(None)
        return timestamp_series
    
    # Align demand data
    if demand_df is not None:
        print("\nAligning demand data...")
        ts_col = find_timestamp_col(demand_df)
        if ts_col:
            # Ensure timestamp is datetime
            demand_df = demand_df.copy()
            demand_df[ts_col] = pd.to_datetime(demand_df[ts_col])
            demand_df[ts_col] = remove_timezone(demand_df[ts_col])
            
            # Filter to target year
            demand_year = demand_df[demand_df[ts_col].dt.year == target_year].copy()
            demand_year['timestamp'] = demand_year[ts_col]
            
            # Merge with reference timestamps
            aligned_demand = aligned_data.merge(
                demand_year[['timestamp', 'electricity']], 
                on='timestamp', 
                how='left'
            )
            
            print(f"  Original shape: {demand_df.shape}")
            print(f"  Filtered to {target_year}: {demand_year.shape}")
            print(f"  Aligned shape: {aligned_demand.shape}")
            print(f"  Missing values: {aligned_demand['electricity'].isna().sum()}")
        else:
            print("  Warning: No timestamp column found")
            aligned_demand = aligned_data.copy()
            aligned_demand['electricity'] = np.nan
    else:
        print("\nDemand data not available")
        aligned_demand = aligned_data.copy()
        aligned_demand['electricity'] = np.nan
    
    # Align spot price data
    if spot_df is not None:
        print("\nAligning spot price data...")
        ts_col = find_timestamp_col(spot_df)
        if ts_col:
            spot_df = spot_df.copy()
            spot_df[ts_col] = pd.to_datetime(spot_df[ts_col])
            spot_df[ts_col] = remove_timezone(spot_df[ts_col])
            
            # Filter to target year
            spot_year = spot_df[spot_df[ts_col].dt.year == target_year].copy()
            spot_year['timestamp'] = spot_year[ts_col]
            
            # Get RRP column (spot price) - check multiple possible names
            if 'RRP' in spot_year.columns:
                price_col = 'RRP'
            elif 'price_per_kwh' in spot_year.columns:
                price_col = 'price_per_kwh'
            elif 'spot_price_kwh' in spot_year.columns:
                price_col = 'spot_price_kwh'
            else:
                # Find any column with 'price' in the name
                price_cols = [col for col in spot_year.columns if 'price' in col.lower() or 'rrp' in col.lower()]
                if price_cols:
                    price_col = price_cols[0]
                    print(f"  Using price column: {price_col}")
                else:
                    print(f"  Warning: No price column found. Available columns: {list(spot_year.columns)}")
                    aligned_spot = aligned_data.copy()
                    aligned_spot['spot_price_kwh'] = np.nan
                    price_col = None
            
            if price_col:
                # Merge with reference timestamps
                aligned_spot = aligned_data.merge(
                    spot_year[['timestamp', price_col]], 
                    on='timestamp', 
                    how='left'
                )
                
                # Convert RRP from $/MWh to $/kWh if needed
                if price_col == 'RRP':
                    aligned_spot['spot_price_kwh'] = aligned_spot['RRP'] / 1000
                    aligned_spot = aligned_spot.drop('RRP', axis=1)
                else:
                    # Assume it's already in $/kWh or rename it
                    aligned_spot['spot_price_kwh'] = aligned_spot[price_col]
                    if price_col != 'spot_price_kwh':
                        aligned_spot = aligned_spot.drop(price_col, axis=1)
                
                # Fill the first timestamp's missing value with the next available value
                if pd.isna(aligned_spot.loc[0, 'spot_price_kwh']):
                    first_valid_value = aligned_spot['spot_price_kwh'].dropna().iloc[0]
                    aligned_spot.loc[0, 'spot_price_kwh'] = first_valid_value
                    print(f"  Filled first timestamp with next available value: {first_valid_value}")
                
                print(f"  Original shape: {spot_df.shape}")
                print(f"  Filtered to {target_year}: {spot_year.shape}")
                print(f"  Aligned shape: {aligned_spot.shape}")
                print(f"  Missing values: {aligned_spot['spot_price_kwh'].isna().sum()}")
        else:
            print("  Warning: No timestamp column found")
            aligned_spot = aligned_data.copy()
            aligned_spot['spot_price_kwh'] = np.nan
    else:
        print("\nSpot price data not available")
        aligned_spot = aligned_data.copy()
        aligned_spot['spot_price_kwh'] = np.nan
    
    # Align grid price data
    if grid_prices_df is not None:
        print("\nAligning grid price data...")
        ts_col = find_timestamp_col(grid_prices_df)
        if ts_col:
            grid_prices_df = grid_prices_df.copy()
            grid_prices_df[ts_col] = pd.to_datetime(grid_prices_df[ts_col])
            grid_prices_df[ts_col] = remove_timezone(grid_prices_df[ts_col])
            
            # Filter to target year
            grid_year = grid_prices_df[grid_prices_df[ts_col].dt.year == target_year].copy()
            grid_year['timestamp'] = grid_year[ts_col]
            
            # Get price column - check multiple possible names
            if 'Price ($/kWh)' in grid_year.columns:
                price_col = 'Price ($/kWh)'
            elif 'price' in grid_year.columns:
                price_col = 'price'
            elif 'retail_price_kwh' in grid_year.columns:
                price_col = 'retail_price_kwh'
            else:
                # Find any column with 'price' in the name
                price_cols = [col for col in grid_year.columns if 'price' in col.lower()]
                if price_cols:
                    price_col = price_cols[0]
                    print(f"  Using price column: {price_col}")
                else:
                    print(f"  Warning: No price column found. Available columns: {list(grid_year.columns)}")
                    aligned_grid = aligned_data.copy()
                    aligned_grid['retail_price_kwh'] = np.nan
                    price_col = None
            
            if price_col:
                # Merge with reference timestamps
                aligned_grid = aligned_data.merge(
                    grid_year[['timestamp', price_col]], 
                    on='timestamp', 
                    how='left'
                )
                
                # Rename to standard column name
                if price_col != 'retail_price_kwh':
                    aligned_grid = aligned_grid.rename(columns={price_col: 'retail_price_kwh'})
                
                print(f"  Original shape: {grid_prices_df.shape}")
                print(f"  Filtered to {target_year}: {grid_year.shape}")
                print(f"  Aligned shape: {aligned_grid.shape}")
                print(f"  Missing values: {aligned_grid['retail_price_kwh'].isna().sum()}")
        else:
            print("  Warning: No timestamp column found")
            aligned_grid = aligned_data.copy()
            aligned_grid['retail_price_kwh'] = np.nan
    else:
        print("\nGrid price data not available")
        aligned_grid = aligned_data.copy()
        aligned_grid['retail_price_kwh'] = np.nan
    
    # Align solar data
    if solar_df is not None:
        print("\nAligning solar generation data...")
        ts_col = find_timestamp_col(solar_df)
        if ts_col:
            solar_df = solar_df.copy()
            solar_df[ts_col] = pd.to_datetime(solar_df[ts_col])
            solar_df[ts_col] = remove_timezone(solar_df[ts_col])
            
            # Filter to target year
            solar_year = solar_df[solar_df[ts_col].dt.year == target_year].copy()
            solar_year['timestamp'] = solar_year[ts_col]
            
            # Get solar columns
            solar_cols = ['north_solar_capacity_kw_per_kw', 'east_solar_capacity_kw_per_kw',
                         'south_solar_capacity_kw_per_kw', 'west_solar_capacity_kw_per_kw']
            available_cols = [col for col in solar_cols if col in solar_year.columns]
            
            if available_cols:
                # Merge with reference timestamps
                aligned_solar = aligned_data.merge(
                    solar_year[['timestamp'] + available_cols], 
                    on='timestamp', 
                    how='left'
                )
                
                # Fill missing values with 0 for solar columns
                for col in available_cols:
                    aligned_solar[col] = aligned_solar[col].fillna(0)
                    print(f"  Filled missing values in {col} with 0")
                
                print(f"  Original shape: {solar_df.shape}")
                print(f"  Filtered to {target_year}: {solar_year.shape}")
                print(f"  Aligned shape: {aligned_solar.shape}")
                print(f"  Solar columns: {available_cols}")
                for col in available_cols:
                    print(f"  Missing values in {col}: {aligned_solar[col].isna().sum()}")
            else:
                print("  Warning: No solar capacity columns found")
                aligned_solar = aligned_data.copy()
        else:
            print("  Warning: No timestamp column found")
            aligned_solar = aligned_data.copy()
    else:
        print("\nSolar data not available")
        aligned_solar = aligned_data.copy()
    
    # Summary
    print(f"\n{'='*80}")
    print("ALIGNMENT SUMMARY")
    print("="*80)
    print(f"Target year: {target_year}")
    print(f"Reference intervals: {len(reference_timestamps)}")
    print(f"\nAligned dataframe shapes:")
    print(f"  Demand: {aligned_demand.shape}")
    print(f"  Spot prices: {aligned_spot.shape}")
    print(f"  Grid prices: {aligned_grid.shape}")
    print(f"  Solar generation: {aligned_solar.shape}")
    
    return aligned_demand, aligned_spot, aligned_grid, aligned_solar


# Specify the target year for alignment
alignment_year = start_year  # Use the configured start_year, or set manually

# Run alignment and replace original dataframes
demand_df, spot_df, grid_prices_df, solar_df = align_dataframes_by_year(
    demand_df, spot_df, grid_prices_df, solar_df, alignment_year
)

# Display aligned data
print(f"\n\n{'='*80}")
print("ALIGNED DATAFRAMES")
print("="*80)

print("\n--- Demand DataFrame ---")
print(demand_df.head(10))
print(f"\nData range: {demand_df['timestamp'].min()} to {demand_df['timestamp'].max()}")
print(f"Shape: {demand_df.shape}")

print("\n--- Spot Prices DataFrame ---")
print(spot_df.head(10))
print(f"\nData range: {spot_df['timestamp'].min()} to {spot_df['timestamp'].max()}")
print(f"Shape: {spot_df.shape}")

print("\n--- Grid Prices DataFrame ---")
print(grid_prices_df.head(10))
print(f"\nData range: {grid_prices_df['timestamp'].min()} to {grid_prices_df['timestamp'].max()}")
print(f"Shape: {grid_prices_df.shape}")

print("\n--- Solar Generation DataFrame ---")
print(solar_df.head(10))
print(f"\nData range: {solar_df['timestamp'].min()} to {solar_df['timestamp'].max()}")
print(f"Shape: {solar_df.shape}")

print("ALIGNMENT COMPLETE!")
print("All dataframes have been aligned and replaced with aligned versions.")

ALIGNING DATAFRAMES TO YEAR 2023

Reference timestamp range:
  Start: 2023-01-01 00:00:00
  End: 2023-12-31 23:55:00
  Total intervals: 105120

Aligning demand data...
  Original shape: (105120, 2)
  Filtered to 2023: (105120, 2)
  Aligned shape: (105120, 2)
  Missing values: 0

Aligning spot price data...
  Filled first timestamp with next available value: 0.12951
  Original shape: (105120, 6)
  Filtered to 2023: (105119, 6)
  Aligned shape: (105120, 2)
  Missing values: 0

Aligning grid price data...
  Original shape: (105120, 2)
  Filtered to 2023: (105120, 3)
  Aligned shape: (105120, 2)
  Missing values: 0

Aligning solar generation data...
  Filled missing values in north_solar_capacity_kw_per_kw with 0
  Filled missing values in east_solar_capacity_kw_per_kw with 0
  Filled missing values in south_solar_capacity_kw_per_kw with 0
  Filled missing values in west_solar_capacity_kw_per_kw with 0
  Original shape: (210240, 5)
  Filtered to 2023: (105120, 5)
  Aligned shape: (105120, 